In [ ]:
USE_SAME_FEW_SHOTS=True

# INSTALLAZIONE E IMPORT LIBRERIE

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

output_data_path = base_path + "/../Llama-2/data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"

In [ ]:
nbit = 4
llamasize = '7B'
# llamasize = '13B'
temp = 0.1
# outdir = 'data/output/2023.10.20-2_LLama2_langchain/'
# outdir = 'data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/'
# outfilename = outdir+f'sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
# outfilename

In [ ]:
#!pip install --upgrade pip
!pip install transformers # Download e caricamento modelli
!pip install bitsandbytes # Quantizzazione
!pip install accelerate

In [ ]:
# Installazione "permanente"
!pip install sentencepiece && touch ~/.ipython/profile_default/startup/00-first.py && echo "import <sentencepiece>" >> ~/.ipython/profile_default/startup/00-first.py

In [ ]:
!pip install pillow==10.0.1
!pip install langchain[llms]

In [ ]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [ ]:
torch.cuda.empty_cache()

# LOGIN DA HUGGING FACE

In [ ]:
!huggingface-cli login --token xxx

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# DOWNLOAD E CARICAMENTO DEL MODELLO

In [ ]:
# pip install --upgrade bitsandbytes

In [ ]:
model = LlamaForCausalLM.from_pretrained(  # Caricamento del modello usando la libreria dedicata. In questo caso, "ForCausal" si riferisce al modello per la generazione di testo
    model_id, # Nome del modello precedentemente impostato
    load_in_4bit = True, # Quantizzazione in 4 bit (per 8 bit usare il parametro load_in_8bit)
    torch_dtype = torch.float16 # Impostare questa codifica aiuta il modello in fase di generazione, in quanto il prompt è tipicamente float16
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Tue Nov 21 17:10:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    26W /  70W |   4873MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm

In [ ]:
torch.cuda.empty_cache()

## TEST

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # max number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
# data = pd.read_excel("data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx")
# data = pd.read_excel("../data/input/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [ ]:
data.shape

(1020, 8)

In [ ]:
data.groupby('sdg').count()['text']

sdg
0     60
1     60
2     60
3     60
4     60
5     60
6     60
7     60
8     60
9     60
10    60
11    60
12    60
13    60
14    60
15    60
16    60
Name: text, dtype: int64

In [ ]:
data_sel = data

In [ ]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(3)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000


In [ ]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

print(few_shots)

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Output: "---SDG 12---"

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight again

In [ ]:
template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are 3 examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
{text}
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [ ]:
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%%time
results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

CPU times: user 59min 12s, sys: 18min 49s, total: 1h 18min 2s
Wall time: 1h 18min 57s


In [ ]:
results_processed = pd.DataFrame(results, columns=['result'])
results_processed

,result
0,---SDG 9---
1,---SDG 8---
2,---SDG 16---
3,---SDG 8---
4,---SDG 17---\n\nThe text you provided can be c...
...,...
1015,---SDG 16---
1016,The text you provided can be classified under ...
1017,---SDG 6---
1018,---SDG 8---


In [ ]:
results_processed['label'] = results_processed['result'].str.extract(r'SDG (\d+)')
results_processed

,result,label
0,---SDG 9---,9
1,---SDG 8---,8
2,---SDG 16---,16
3,---SDG 8---,8
4,---SDG 17---\n\nThe text you provided can be c...,17
...,...,...
1015,---SDG 16---,16
1016,The text you provided can be classified under ...,17
1017,---SDG 6---,6
1018,---SDG 8---,8


In [ ]:
final = data_sel.merge(results_processed, left_index=True, right_index=True)
final

,Unnamed: 0,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,result,label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,---SDG 9---,9
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,---SDG 8---,8
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,---SDG 8---,8
4,4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,---SDG 17---\n\nThe text you provided can be c...,17
...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,---SDG 16---,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,The text you provided can be classified under ...,17
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,---SDG 6---,6
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,---SDG 8---,8


In [ ]:
final.to_excel(output_data_path+"/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot.xlsx")

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.75      0.20      0.32        60
           2       1.00      0.20      0.33        60
           3       0.87      0.65      0.74        60
           4       0.88      0.37      0.52        60
           5       0.97      0.47      0.63        60
           6       0.91      0.52      0.66        60
           7       0.94      0.27      0.42        60
           8       0.16      0.88      0.28        60
           9       0.36      0.55      0.43        60
          10       0.39      0.18      0.25        60
          11       0.63      0.52      0.57        60
          12       0.45      0.72      0.55        60
          13       0.68      0.47      0.55        60
          14       0.93      0.42      0.57        60
          15       0.67      0.72      0.69        60
          16       0.40      0.57      0.47        60
          17     

In [ ]:
classification_report_filename = output_data_path+f'/sdg_classification_llama_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}_classification_report.xlsx'
classification_report_filename

'/content/gdrive/MyDrive/Colab/SDG/data/../Llama-2/data/output/2023.11.21-1_LLama2_7B-chat-hf_4-bit_float-16_3-few-shot/sdg_classification_llama_1020-texts_temp-0.1_nbit-4_llamasize-7B_classification_report.xlsx'

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [ ]:
test = pd.read_excel(classification_report_filename)
test

,Unnamed: 0,precision,recall,f1-score,support
0,0,0.000000,0.000000,0.000000,60.000000
1,1,0.750000,0.200000,0.315789,60.000000
2,2,1.000000,0.200000,0.333333,60.000000
3,3,0.866667,0.650000,0.742857,60.000000
4,4,0.880000,0.366667,0.517647,60.000000
5,5,0.965517,0.466667,0.629213,60.000000
6,6,0.911765,0.516667,0.659574,60.000000
7,7,0.941176,0.266667,0.415584,60.000000
8,8,0.164087,0.883333,0.276762,60.000000
9,9,0.358696,0.550000,0.434211,60.000000
